In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-06-09 08:11:21.996668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 08:11:21.996778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 08:11:22.125958: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/threat-classiffication/Hate Speech - Labeled.xlsx
/kaggle/input/threat-classiffication/Neutral - Labeled.xlsx
/kaggle/input/threat-classiffication/All Threat - Labeled.xlsx
/kaggle/input/threat-hate-neutral/Normal Only Dataset.xlsx
/kaggle/input/threat-hate-neutral/Hate Only Spreadsheet.xlsx
/kaggle/input/threat-hate-neutral/Threat Only Dataset.xlsx


In [2]:
df_threat = pd.read_excel('/kaggle/input/threat-hate-neutral/Threat Only Dataset.xlsx')
df_threat = df_threat.dropna()
# df_threat = df_threat.iloc[:3000]

df_hate = pd.read_excel('/kaggle/input/threat-hate-neutral/Hate Only Spreadsheet.xlsx')
df_hate = df_hate.dropna()
df_hate = df_hate.iloc[:2500]

df_neutral = pd.read_excel('/kaggle/input/threat-hate-neutral/Normal Only Dataset.xlsx')
df_neutral = df_neutral.dropna()
df_neutral=df_neutral.iloc[:2500]

In [3]:
print(df_threat.isna().sum())
print(df_neutral.isna().sum())
print(df_hate.isna().sum())

text     0
label    0
dtype: int64
text     0
label    0
dtype: int64
text     0
label    0
dtype: int64


In [4]:
# merging the two dataframes
frame=[df_hate, df_neutral, df_threat]
merge=pd.concat(frame)

In [5]:
merge.label.unique()

array(['Hate', 'Free', 'threat'], dtype=object)

In [6]:
merge['labels'] = merge['label'].astype('category').cat.codes
merge['labels'].unique()

array([1, 0, 2], dtype=int8)

In [7]:
merge

,text,label,labels
0,መካን ሴት ከማንም ጋር መጋባት የለባትም ምክንያቱም ይህ ፍትሃዊ አይደለም,Hate,1
1,ብሪተኖች ፋሺስቶች እንዳልሆኑ በመስማቴ ታምኛለሁ,Hate,1
2,እንደ አንድ ነገር መለየት እውነት አያደርገውም። ሰዎች በቀዶ ሕክምና ራሳ...,Hate,1
3,ሁሉም ሴቶች ቢሰበሰቡ በደም ገንዳ ውስጥ እንሰጥም ነበር,Hate,1
4,ቲቪ ጥቁሮችን እንደ ጠበኛ ወንጀለኞች በትክክል ይወክላል,Hate,1
...,...,...,...
1424,ሴቶች መደፈር አለባቸው,threat,2
1425,ካላር ጭንቅላት የሚቆርጥ ቡድን እደግፋለሁ,threat,2
1426,ዳይክ የሌለባትን አለም አስብ አሁን እናድርገው,threat,2
1427,የኔ ነጭ ላፕቶፕ ከሁሉም ነጭ ሴቶች ይበልጣል,threat,2


In [8]:
merge["labels"].nunique()

3

In [9]:
# mixing each categories and thier labels
dataframe=merge.sample(frac=1)
dataframe

,text,label,labels
1656,ሚኪ ሁሌም የምታመነጫቸው ሀሳቦች አዳዲስ አንተ ያማራ ብርቅዬ ልጅ ነህ ያ...,Free,0
549,ለመላው የኦርቶዶክስ እምነት ተከታዪች በሙሉ ሀይላችንም ጉልበታችንም ፈጣራ...,Free,0
2436,በኦሮሚያ ክልል የተፈጠረው ግጭት ተከትሎ ንፁሀን ወገኖቻችን ሞት መንግስት...,Hate,1
816,ሙስሊሞች ቫይረስ ናቸው,threat,2
2369,በዚህ ሰው ቦታ ብሆን የኤርትራ ሪፈረንደም ዳግም እንዲካሄድ አደርጋሎሁ ከ...,Hate,1
...,...,...,...
1025,ከዚህ በመለስ የሀገሪቱ ሚሊዮን ህዝብ ሙስሊሙ ይህን ባንዲራ ይፀየፏል ሲቀ...,Hate,1
1193,አንድ ነገር ትንፍስ እላለው ትይና ጉድሽን ነግሬ ቤትሽን እበጠብጠዋለው,threat,2
504,ምናልባት ራሷን ተኩሳ ሊሆን ይችላል,threat,2
912,ታደለ ሀክ ተደረገ እንዴነው ክርሲቲያን አማኝ የሚል ጠፋ ግራ አጋባህን የ...,Free,0


In [10]:
dataframe['word_len'] = dataframe['text'].str.split().str.len()
print(dataframe['word_len'].mean())

12.986156478456993


In [11]:
del dataframe['label']

In [12]:
dataframe

,text,labels,word_len
1656,ሚኪ ሁሌም የምታመነጫቸው ሀሳቦች አዳዲስ አንተ ያማራ ብርቅዬ ልጅ ነህ ያ...,0,14
549,ለመላው የኦርቶዶክስ እምነት ተከታዪች በሙሉ ሀይላችንም ጉልበታችንም ፈጣራ...,0,42
2436,በኦሮሚያ ክልል የተፈጠረው ግጭት ተከትሎ ንፁሀን ወገኖቻችን ሞት መንግስት...,1,26
816,ሙስሊሞች ቫይረስ ናቸው,2,3
2369,በዚህ ሰው ቦታ ብሆን የኤርትራ ሪፈረንደም ዳግም እንዲካሄድ አደርጋሎሁ ከ...,1,24
...,...,...,...
1025,ከዚህ በመለስ የሀገሪቱ ሚሊዮን ህዝብ ሙስሊሙ ይህን ባንዲራ ይፀየፏል ሲቀ...,1,23
1193,አንድ ነገር ትንፍስ እላለው ትይና ጉድሽን ነግሬ ቤትሽን እበጠብጠዋለው,2,9
504,ምናልባት ራሷን ተኩሳ ሊሆን ይችላል,2,5
912,ታደለ ሀክ ተደረገ እንዴነው ክርሲቲያን አማኝ የሚል ጠፋ ግራ አጋባህን የ...,0,16


In [13]:
#normalizarion
import re
#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalization(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48

In [14]:
# apply normalization and special characters removal
dataframe['text']=dataframe['text'].apply(lambda x: normalization(x))

In [15]:
# special symbol removal
dataframe['text']=dataframe['text'].str.replace('[^\w\s]','',regex=True)

In [16]:
# removing digits from the dataset
dataframe['text']=dataframe['text'].str.replace('\d+','',regex=True)

In [17]:
dataframe

,text,labels,word_len
1656,ሚኪ ሁሌም የምታመነጫቸው ሀሳቦች አዳዲስ አንተ ያማራ ብርቅዬ ልጅ ነህ ያ...,0,14
549,ለመላው የኦርቶዶክስ እምነት ተከታዪች በሙሉ ሀይላችንም ጉልበታችንም ፈጣራ...,0,42
2436,በኦሮሚያ ክልል የተፈጠረው ግጭት ተከትሎ ንፁሀን ወገኖቻችን ሞት መንግስት...,1,26
816,ሙስሊሞች ቫይረስ ናቸው,2,3
2369,በዚህ ሰው ቦታ ብሆን የኤርትራ ሪፈረንደም ዳግም እንዲካሄድ አደርጋሎሁ ከ...,1,24
...,...,...,...
1025,ከዚህ በመለስ የሀገሪቱ ሚሊዮን ህዝብ ሙስሊሙ ይህን ባንዲራ ይፀየፏል ሲቀ...,1,23
1193,አንድ ነገር ትንፍስ እላለው ትይና ጉድሽን ነግሬ ቤትሽን እበጠብጠዋለው,2,9
504,ምናልባት ራሷን ተኩሳ ሊሆን ይችላል,2,5
912,ታደለ ሀክ ተደረገ እንዴነው ክርሲቲያን አማኝ የሚል ጠፋ ግራ አጋባህን የ...,0,16


In [18]:
# train test split
from sklearn.model_selection import train_test_split
train_val_df, test_dataset = train_test_split(dataframe, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.10, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (4628, 3)
Validation dataset shape:  (515, 3)
Testing dataset shape:  (1286, 3)


In [19]:
msk = np.random.rand(len(dataframe)) < 0.8
train_dataset = dataframe[msk]
test_dataset = dataframe[~msk]

In [20]:
test_dataset

,text,labels,word_len
2369,በዚህ ሰው ቦታ ብሆን የኤርትራ ሪፈረንደም ዳግም እንዲካሄድ አደርጋሎሁ ከ...,1,24
2448,የነ ቶሎ ቶሎ ቤት ግድግዳው ሰምበሌጥ ተራርጠሽ ስታጨበጪቢ አልነበረም አነ...,1,73
2276,አስቱ ስለእውነት ስለኢትዮጲያ ጥሩ የሚያስብ ሀሳቡን ቢመለከት ልደቱ የሚሞ...,1,18
1708,እባካችሁ ፍርዱን ለፈጣሪ ስጡ,0,4
359,እንኳን ኢትዮጵያን በመሰለች ደሀ አገር በትላልቅ አገሮች እንኳ የገንዘብ ...,1,20
...,...,...,...
1809,አንተ ራስህ ገዱ አስተዋይ ነህ መልካም እይታ እኔን የገረመኝ የሆኑን ሰዎ...,0,37
566,እኔስ ሁለየ የማስባቸው አቅም አጣሁ,1,5
1412,እነ እስክድርና እናንተ መች የህዝብ ስሜት ይገባቸዋል ለነሱ ስልጣንና ስል...,1,21
622,እምነት የጎደለው ከብት ክዚህ እርኩስ ተግባሩ እጁን ቢያነሳ ይሻለዋል,0,9


In [21]:
!pip install datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [22]:
from datasets import Dataset


In [23]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)
train_dataset=Dataset.from_pandas(train_dataset)
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [24]:
(test_dataset)

Dataset({
    features: ['text', 'labels', 'word_len', '__index_level_0__'],
    num_rows: 1295
})

In [25]:
(evaluation_dataset)

Dataset({
    features: ['text', 'labels', 'word_len', '__index_level_0__'],
    num_rows: 515
})

In [26]:
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [27]:
import datasets
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [28]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluate'].num_rows

In [29]:
from transformers import AutoTokenizer

In [30]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [31]:
def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)

In [32]:
tokenized_datasets = main_dataset.map(tokenize_function)

Map:   0%|          | 0/5134 [00:00<?, ? examples/s]

Map:   0%|          | 0/1295 [00:00<?, ? examples/s]

Map:   0%|          | 0/515 [00:00<?, ? examples/s]

In [33]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [34]:
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm

In [35]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [36]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [37]:
small_eval_dataset

Dataset({
    features: ['labels', 'word_len', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 515
})

In [38]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

In [39]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=3)

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
import numpy as np
from datasets import load_metric
#metric = load_metric("f1","accuracy")

In [41]:
! pip install -U accelerate
! pip install -U transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [43]:
import accelerate
accelerate.__version__

'0.30.1'

In [44]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [45]:
def custom_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [46]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy='epoch',
   save_strategy='epoch',
   num_train_epochs=8,
   # total number of training epochs
   learning_rate=1e-5,
   per_device_train_batch_size=4,  # batch size per device during training
   per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=1000,                # number of warmup steps for learning rate
    load_best_model_at_end= True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=20,

   )

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=custom_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
)

In [48]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.734700,0.782878,0.675315,0.627184,0.603344,0.627184
2,0.718200,0.518106,0.774791,0.768932,0.770836,0.768932
3,0.582400,0.388380,0.844496,0.819417,0.816713,0.819417
4,0.508000,0.259230,0.907639,0.906796,0.906744,0.906796
5,0.183900,0.262209,0.929909,0.930097,0.929843,0.930097
6,0.195900,0.285157,0.941608,0.941748,0.941600,0.941748
7,0.183800,0.309685,0.946181,0.945631,0.945427,0.945631
8,0.121400,0.334672,0.944383,0.943689,0.943463,0.943689


...

/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/recall/recall.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect 

TrainOutput(global_step=5136, training_loss=0.4238160080929226, metrics={'train_runtime': 3096.5864, 'train_samples_per_second': 13.264, 'train_steps_per_second': 1.659, 'total_flos': 1.0806594292924416e+16, 'train_loss': 0.4238160080929226, 'epoch': 8.0})

In [49]:
trainer.evaluate(small_test_dataset)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/recall/recall.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric fro

{'eval_loss': 0.7967183589935303,
 'eval_precision': 0.715379606884165,
 'eval_recall': 0.7135135135135136,
 'eval_f1': 0.7095801912784023,
 'eval_accuracy': 0.7135135135135136,
 'eval_runtime': 24.3544,
 'eval_samples_per_second': 53.173,
 'eval_steps_per_second': 3.326,
 'epoch': 8.0}

In [61]:
trainer.save_model('CustomModel')

In [62]:
model_2 = AutoModelForSequenceClassification.from_pretrained("CustomModel")

In [64]:
# text = "That was good point"
text = "እገልሃለሁ"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.00413685, 0.01703663, 0.9788265 ]], dtype=float32)

In [65]:

!pip install huggingface_hub

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [66]:
from huggingface_hub import notebook_login

notebook_login()

In [67]:
model.push_to_hub("Rediet03/Threat-Detection-Finetuned-Bert")


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rediet03/Threat-Detection-Finetuned-Bert/commit/9e61b1c36524ba800f74ee7657b7cfa38f13dc19', commit_message='Upload BertForSequenceClassification', commit_description='', oid='9e61b1c36524ba800f74ee7657b7cfa38f13dc19', pr_url=None, pr_revision=None, pr_num=None)

In [68]:
model_3 = AutoModelForSequenceClassification.from_pretrained('Rediet03/Threat-Detection-Finetuned-Bert', )

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [70]:
model_3

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [80]:
# text = "That was good point"
text = "እገልሃለሁ እኔን አያርገኝ"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.00216455, 0.01035448, 0.98748094]], dtype=float32)

In [75]:
# from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification
# import os

# # Define the directory where you want to save the model
# output_dir = "/kaggle/working/saved_model"  # Change this to your desired directory

# # Create the directory if it does not exist
# os.makedirs(output_dir, exist_ok=True)

# # Save the tokenizer
# tokenizer.save_pretrained(output_dir)

# # Save the model
# model.save_pretrained(output_dir)

# # Save the training arguments
# training_args.save(output_dir)

# # Now you can load the saved model, tokenizer, and training arguments like this:
# tokenizer = AutoTokenizer.from_pretrained(output_dir)
# model = AutoModelForSequenceClassification.from_pretrained(output_dir)
# training_args = TrainingArguments.load(output_dir)


In [51]:
# pt_save_directory = "/kaggle/working/model"
# tokenizer.save_pretrained(pt_save_directory)
# model.save_pretrained(pt_save_directory)

In [52]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import zipfile
# import os

# # Define a simple model
# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
#         self.fc = nn.Linear(10, 2)

#     def forward(self, x):
#         return self.fc(x)

# # Initialize the model, criterion and optimizer
# # model = SimpleModel()
# # criterion = nn.CrossEntropyLoss()
# # optimizer = optim.Adam(model.parameters())

# # Assume you've trained the model here

# # Save the model
# model_path = '/kaggle/working/model_1.safetensors'
# torch.save(model.state_dict(), model_path)

# # Compress the model file
# zip_path = '/kaggle/working/model.zip'
# with zipfile.ZipFile(zip_path, 'w') as zipf:
#     zipf.write(model_path, os.path.basename(model_path))

# # Verify the compressed file
# print(f"Original model size: {os.path.getsize(model_path)} bytes")
# print(f"Compressed model size: {os.path.getsize(zip_path)} bytes")


In [53]:
# import zipfile
# import os

# def zip_directory(folder_path, zip_path):
#     with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, dirs, files in os.walk(folder_path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 arcname = os.path.relpath(file_path, folder_path)
#                 zipf.write(file_path, arcname)

# # Define the path of the directory to be zipped and the zip file name
# directory_to_zip = '/kaggle/working/model'  # Change this to your folder
# output_zip_path = '/kaggle/working/model.zip'

# # Zip the directory
# zip_directory(directory_to_zip, output_zip_path)

# # Verify the zip file
# print(f"Zip file created at: {output_zip_path}")
# print(f"Zip file size: {os.path.getsize(output_zip_path)} bytes")


In [54]:
# !pip install pydrive


In [55]:
# from IPython.display import FileLink
# FileLink(r'/kaggle/working/pt_save_pretrained/model.safetensors')

In [56]:
# import IPython
# import requests

# # Replace 'model.safetensors' with the path to your file
# file_path = "https://kkb-production.jupyter-proxy.kaggle.net/kaggle/working/pt_save_pretrained/model.safetensors"

# # Function to download the file
# def download_file(file_path):
#     response = requests.get(file_path)
#     file_name = file_path.split('/')[-1]
#     with open(file_name, 'wb') as file:
#         file.write(response.content)
#     return file_name

# # Download the file
# downloaded_file = download_file(file_path)

# # Display a link to the downloaded file
# IPython.display.FileLink(downloaded_file)

In [57]:
# from transformers import pipeline

# classifier = pipeline('text-classification', model="/kaggle/working/model")
# classifier("ዉሃ እወድሃለሁ")

In [58]:
# classifier("እገልሃለሁ ግን እወድሃለሁም")

In [59]:
# classifier("ዉሃ እወድሃለሁ")